СМОТР ДАННЫХ

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

КОММЕНТАРИИ

In [12]:
df = pd.read_excel('inputDataComments.xlsx')
df

,commentID,message,masterID,requestID
0,1,Всё сделаем!,2,1
1,2,Всё сделаем!,3,2
2,3,Починим в момент.,3,3


ОБРАЩЕНИЯ

In [11]:
df = pd.read_excel('inputDataRequests.xlsx')
df

,requestID,startDate,climateTechType,climateTechModel,problemDescryption,requestStatus,completionDate,repairParts,masterID,clientID
0,1,2023-06-06,Кондиционер,TCL TAC-12CHSA/TPG-W белый,Не охлаждает воздух,В процессе ремонта,NaN,NaN,2.0,7
1,2,2023-05-05,Кондиционер,Electrolux EACS/I-09HAT/N3_21Y белый,Выключается сам по себе,В процессе ремонта,NaN,NaN,3.0,8
2,3,2022-07-07,Увлажнитель воздуха,Xiaomi Smart Humidifier 2,Пар имеет неприятный запах,Готова к выдаче,2023-01-01,NaN,3.0,9
3,4,2023-08-02,Увлажнитель воздуха,Polaris PUH 2300 WIFI IQ Home,Увлажнитель воздуха продолжает работать при пр...,Новая заявка,NaN,NaN,NaN,8
4,5,2023-08-02,Сушилка для рук,Ballu BAHD-1250,Не работает,Новая заявка,NaN,NaN,NaN,9


ПОЛЬЗОВАТЕЛИ

In [10]:
df = pd.read_excel('inputDataUsers.xlsx')
df

,userID,fio,phone,login,password,type
0,1,Широков Василий Матвеевич,89210563128,login1,pass1,Менеджер
1,2,Кудрявцева Ева Ивановна,89535078985,login2,pass2,Специалист
2,3,Гончарова Ульяна Ярославовна,89210673849,login3,pass3,Специалист
3,4,Гусева Виктория Данииловна,89990563748,login4,pass4,Оператор
4,5,Баранов Артём Юрьевич,89994563847,login5,pass5,Оператор
5,6,Овчинников Фёдор Никитич,89219567849,login6,pass6,Заказчик
6,7,Петров Никита Артёмович,89219567841,login7,pass7,Заказчик
7,8,Ковалева Софья Владимировна,89219567842,login8,pass8,Заказчик
8,9,Кузнецов Сергей Матвеевич,89219567843,login9,pass9,Заказчик
9,10,Беспалова Екатерина Даниэльевна,89219567844,login10,pass10,Специалист


СОЗДАНИЕ БД

In [18]:
import pandas as pd
import sqlite3
import json
from datetime import datetime

def create_combined_table():
    """Создает единую таблицу, объединяющую все данные из XLSX файлов"""
    
    # 1. Загружаем данные из XLSX файлов
    df_requests = pd.read_excel('inputDataRequests.xlsx')
    df_comments = pd.read_excel('inputDataComments.xlsx')
    df_users = pd.read_excel('inputDataUsers.xlsx')
    
    # 2. Объединяем данные
    # Создаем копию запросов
    df_combined = df_requests.copy()
    
    # Добавляем комментарии к запросам
    comments_info = df_comments.set_index('requestID')[['message', 'masterID']]
    # Переименовываем masterID из комментариев, чтобы избежать конфликта имен
    comments_info = comments_info.rename(columns={'masterID': 'comment_masterID'})
    df_combined = df_combined.merge(comments_info, left_on='requestID', right_index=True, how='left')
    
    # Добавляем информацию о клиенте
    client_info = df_users.set_index('userID')[['fio', 'phone', 'login', 'type']]
    client_info = client_info.rename(columns={
        'fio': 'client_fio',
        'phone': 'client_phone',
        'login': 'client_login',
        'type': 'client_type'
    })
    df_combined = df_combined.merge(client_info, left_on='clientID', right_index=True, how='left')
    
    # Добавляем информацию о мастере (из запросов)
    master_info = df_users.set_index('userID')[['fio', 'phone', 'login', 'type']]
    master_info = master_info.rename(columns={
        'fio': 'master_fio',
        'phone': 'master_phone',
        'login': 'master_login',
        'type': 'master_type'
    })
    df_combined = df_combined.merge(master_info, left_on='masterID', right_index=True, how='left')
    
    # 3. Переименовываем столбцы для единообразия
    column_mapping = {
        'requestID': 'request_id',
        'startDate': 'start_date',
        'climateTechType': 'tech_type',
        'climateTechModel': 'tech_model',
        'problemDescryption': 'problem_description',
        'requestStatus': 'request_status',
        'completionDate': 'completion_date',
        'repairParts': 'repair_parts',
        'masterID': 'master_id',
        'clientID': 'client_id',
        'commentID': 'comment_id',
        'message': 'comment_message',
        'comment_masterID': 'comment_master_id'
    }
    df_combined.rename(columns=column_mapping, inplace=True)
    
    # 4. Преобразуем даты в правильный формат
    date_columns = ['start_date', 'completion_date']
    for col in date_columns:
        if col in df_combined.columns:
            df_combined[col] = pd.to_datetime(df_combined[col], errors='coerce')
    
    # 5. Добавляем вычисляемые столбцы
    df_combined['days_in_process'] = None
    for idx, row in df_combined.iterrows():
        if pd.notnull(row['start_date']) and row['request_status'] != 'Новая заявка':
            if pd.notnull(row['completion_date']):
                # Если есть дата завершения
                df_combined.at[idx, 'days_in_process'] = (row['completion_date'] - row['start_date']).days
            else:
                # Если заявка еще в процессе, считаем от текущей даты
                df_combined.at[idx, 'days_in_process'] = (pd.Timestamp.now() - row['start_date']).days
    
    # 6. Добавляем индикатор наличия комментария
    df_combined['has_comment'] = df_combined['comment_message'].notnull()
    
    # 7. Сортируем и добавляем ID
    df_combined.sort_values(['request_id', 'start_date'], inplace=True)
    df_combined.reset_index(drop=True, inplace=True)
    df_combined['id'] = range(1, len(df_combined) + 1)
    
    # 8. Переупорядочиваем столбцы
    columns_order = [
        'id',
        'request_id',
        'start_date',
        'tech_type',
        'tech_model',
        'problem_description',
        'request_status',
        'completion_date',
        'days_in_process',
        'repair_parts',
        'has_comment',
        'comment_message',
        'master_id',
        'master_fio',
        'master_phone',
        'master_login',
        'master_type',
        'client_id',
        'client_fio',
        'client_phone',
        'client_login',
        'client_type',
        'comment_master_id'
    ]
    
    # Выбираем только существующие столбцы
    existing_columns = [col for col in columns_order if col in df_combined.columns]
    
    return df_combined[existing_columns]

def save_all_formats(df_combined):
    """Сохраняет данные во всех форматах"""
    # Excel
    df_combined.to_excel('service_requests_combined.xlsx', index=False)
    print("✓ Excel файл создан: service_requests_combined.xlsx")
    
    # SQLite
    conn = sqlite3.connect('service_requests.db')
    df_combined.to_sql('service_requests', conn, if_exists='replace', index=False)
    
    # Создаем индексы
    cursor = conn.cursor()
    cursor.execute('CREATE INDEX idx_request_id ON service_requests(request_id)')
    cursor.execute('CREATE INDEX idx_client_id ON service_requests(client_id)')
    cursor.execute('CREATE INDEX idx_master_id ON service_requests(master_id)')
    cursor.execute('CREATE INDEX idx_status ON service_requests(request_status)')
    conn.commit()
    conn.close()
    print("✓ База данных SQLite создана: service_requests.db")
    
    # CSV
    df_combined.to_csv('service_requests_combined.csv', index=False, encoding='utf-8-sig', sep=';')
    print("✓ CSV файл создан: service_requests_combined.csv")
    
    # JSON
    data_dict = df_combined.to_dict(orient='records')
    
    # Преобразуем datetime в строки для JSON
    for record in data_dict:
        for key, value in record.items():
            if isinstance(value, pd.Timestamp):
                record[key] = value.strftime('%Y-%m-%d')
            elif pd.isna(value):
                record[key] = None
    
    with open('service_requests_combined.json', 'w', encoding='utf-8') as f:
        json.dump(data_dict, f, ensure_ascii=False, indent=2)
    print("✓ JSON файл создан: service_requests_combined.json")
    
    # Дополнительные отчеты
    # Сводный отчет по статусам заявок
    status_summary = df_combined.groupby('request_status').agg({
        'request_id': 'count',
        'days_in_process': 'mean'
    }).reset_index()
    status_summary.columns = ['Статус заявки', 'Количество заявок', 'Среднее время обработки (дни)']
    status_summary.to_excel('status_summary.xlsx', index=False)
    print("✓ Сводный отчет по статусам: status_summary.xlsx")
    
    # Отчет по мастерам
    if 'master_fio' in df_combined.columns:
        master_summary = df_combined[df_combined['master_id'].notnull()].groupby(['master_id', 'master_fio']).agg({
            'request_id': 'count',
            'days_in_process': 'mean'
        }).reset_index()
        master_summary.columns = ['ID Мастера', 'ФИО Мастера', 'Количество заявок', 'Среднее время выполнения (дни)']
        master_summary.to_excel('master_summary.xlsx', index=False)
        print("✓ Отчет по мастерам: master_summary.xlsx")

def main():
    """Основная функция"""
    print("Объединение данных из XLSX файлов...")
    print("-" * 50)
    
    try:
        # Создаем объединенную таблицу
        df_combined = create_combined_table()
        
        # Показываем информацию о таблице
        print(f"Создана объединенная таблица с {len(df_combined)} строками")
        print(f"Количество столбцов: {len(df_combined.columns)}")
        print("\nСтруктура таблицы:")
        for col in df_combined.columns:
            print(f"  - {col}")
        
        # Показываем первые строки
        print("\nПервые 3 строки данных:")
        print(df_combined.head(3).to_string(index=False))
        
        # Сохраняем во всех форматах
        print("\n" + "-" * 50)
        print("Сохранение данных...")
        save_all_formats(df_combined)
        
        print("\n" + "-" * 50)
        print("ВЫПОЛНЕНО УСПЕШНО!")
        print("\nСозданы файлы:")
        print("1. service_requests_combined.xlsx - Основная таблица в Excel")
        print("2. service_requests.db - База данных SQLite")
        print("3. service_requests_combined.csv - Данные в CSV формате")
        print("4. service_requests_combined.json - Данные в JSON формате")
        print("5. status_summary.xlsx - Сводный отчет по статусам заявок")
        print("6. master_summary.xlsx - Отчет по мастерам")
        
        return df_combined
        
    except FileNotFoundError as e:
        print(f"ОШИБКА: Не найден файл {e}")
        print("\nУбедитесь, что все файлы находятся в той же папке:")
        print("1. inputDataRequests.xlsx")
        print("2. inputDataComments.xlsx")
        print("3. inputDataUsers.xlsx")
        return None
    except Exception as e:
        print(f"ОШИБКА: {e}")
        import traceback
        traceback.print_exc()
        return None

if __name__ == "__main__":
    main()

Объединение данных из XLSX файлов...
--------------------------------------------------
Создана объединенная таблица с 5 строками
Количество столбцов: 23

Структура таблицы:
  - id
  - request_id
  - start_date
  - tech_type
  - tech_model
  - problem_description
  - request_status
  - completion_date
  - days_in_process
  - repair_parts
  - has_comment
  - comment_message
  - master_id
  - master_fio
  - master_phone
  - master_login
  - master_type
  - client_id
  - client_fio
  - client_phone
  - client_login
  - client_type
  - comment_master_id

Первые 3 строки данных:
 id  request_id start_date           tech_type                           tech_model        problem_description     request_status completion_date days_in_process  repair_parts  has_comment   comment_message  master_id                   master_fio  master_phone master_login master_type  client_id                  client_fio  client_phone client_login client_type  comment_master_id
  1           1 2023-06-06         К

ТАБЛИЦА С ВСЕМИ ДАННЫМИ

In [20]:
df = pd.read_excel('service_requests_combined.xlsx')

# Для Jupyter Notebook
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

# Отображаем
display(HTML(df.to_html()))

,id,request_id,start_date,tech_type,tech_model,problem_description,request_status,completion_date,days_in_process,repair_parts,has_comment,comment_message,master_id,master_fio,master_phone,master_login,master_type,client_id,client_fio,client_phone,client_login,client_type,comment_master_id
0,1,1,2023-06-06,Кондиционер,TCL TAC-12CHSA/TPG-W белый,Не охлаждает воздух,В процессе ремонта,NaT,923.0,NaN,True,Всё сделаем!,2.0,Кудрявцева Ева Ивановна,8.953508e+10,login2,Специалист,7,Петров Никита Артёмович,89219567841,login7,Заказчик,2.0
1,2,2,2023-05-05,Кондиционер,Electrolux EACS/I-09HAT/N3_21Y белый,Выключается сам по себе,В процессе ремонта,NaT,955.0,NaN,True,Всё сделаем!,3.0,Гончарова Ульяна Ярославовна,8.921067e+10,login3,Специалист,8,Ковалева Софья Владимировна,89219567842,login8,Заказчик,3.0
2,3,3,2022-07-07,Увлажнитель воздуха,Xiaomi Smart Humidifier 2,Пар имеет неприятный запах,Готова к выдаче,2023-01-01,178.0,NaN,True,Починим в момент.,3.0,Гончарова Ульяна Ярославовна,8.921067e+10,login3,Специалист,9,Кузнецов Сергей Матвеевич,89219567843,login9,Заказчик,3.0
3,4,4,2023-08-02,Увлажнитель воздуха,Polaris PUH 2300 WIFI IQ Home,Увлажнитель воздуха продолжает работать при предельном снижении уровня воды,Новая заявка,NaT,NaN,NaN,False,NaN,NaN,NaN,NaN,NaN,NaN,8,Ковалева Софья Владимировна,89219567842,login8,Заказчик,NaN
4,5,5,2023-08-02,Сушилка для рук,Ballu BAHD-1250,Не работает,Новая заявка,NaT,NaN,NaN,False,NaN,NaN,NaN,NaN,NaN,NaN,9,Кузнецов Сергей Матвеевич,89219567843,login9,Заказчик,NaN


ДИАГРАММЫ

In [23]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib
from matplotlib.backends.backend_pdf import PdfPages
import seaborn as sns
import numpy as np
from datetime import datetime, timedelta
import os
import warnings
warnings.filterwarnings('ignore')

# Устанавливаем стиль для красивых графиков
plt.style.use('seaborn-v0_8-darkgrid')
matplotlib.rcParams['font.size'] = 10
matplotlib.rcParams['axes.titlesize'] = 12
matplotlib.rcParams['axes.labelsize'] = 11
matplotlib.rcParams['font.family'] = 'DejaVu Sans'  # Улучшенная поддержка кириллицы

def create_service_dashboard_pdf(excel_file_path, output_pdf_path=None):
    """
    Создает PDF файл с диаграммами на основе данных из Excel файла с сервисными заявками
    
    Параметры:
    ----------
    excel_file_path : str
        Путь к Excel файлу с данными сервисных заявок
    output_pdf_path : str, optional
        Путь для сохранения PDF файла. Если не указан, генерируется автоматически.
    
    Возвращает:
    -----------
    str
        Абсолютный путь к созданному PDF файлу
    """
    
    try:
        # Загружаем данные из Excel
        print(f"Загрузка данных из файла: {excel_file_path}")
        df = pd.read_excel(excel_file_path, engine='openpyxl')
        
        print(f"Данные загружены: {df.shape[0]} строк, {df.shape[1]} столбцов")
        print(f"Столбцы: {list(df.columns)}")
        
        # Преобразуем даты
        df['start_date'] = pd.to_datetime(df['start_date'], errors='coerce')
        df['completion_date'] = pd.to_datetime(df['completion_date'], errors='coerce')
        
        # Если путь для сохранения не указан, генерируем автоматически
        if output_pdf_path is None:
            timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
            output_pdf_path = f"service_dashboard_{timestamp}.pdf"
        
        # Убедимся, что путь абсолютный и нормализованный
        output_pdf_path = os.path.abspath(output_pdf_path)
        
        # Создаем директорию, если она не существует
        output_dir = os.path.dirname(output_pdf_path)
        if output_dir and not os.path.exists(output_dir):
            os.makedirs(output_dir)
        
        print(f"PDF файл будет сохранен как: {output_pdf_path}")
        
        # Создаем PDF файл
        with PdfPages(output_pdf_path) as pdf:
            print(f"Создание PDF файла...")
            
            # Титульная страница
            fig, ax = plt.subplots(figsize=(11, 8.5))
            ax.axis('off')
            
            # Заголовок
            title_text = "Аналитический отчет по сервисным заявкам"
            ax.text(0.5, 0.7, title_text, 
                   fontsize=24, fontweight='bold', 
                   ha='center', va='center', transform=ax.transAxes)
            
            # Подзаголовок
            excel_filename = os.path.basename(excel_file_path)
            subtitle_text = f"На основе данных из файла: {excel_filename}"
            ax.text(0.5, 0.6, subtitle_text, 
                   fontsize=14, 
                   ha='center', va='center', transform=ax.transAxes)
            
            # Информация о данных
            if 'request_status' in df.columns:
                completed_requests = df[df['request_status'].str.contains('Готова|Завершена', case=False, na=False)].shape[0]
                in_progress_requests = df[df['request_status'].str.contains('В процессе|ремонта', case=False, na=False)].shape[0]
                new_requests = df[df['request_status'].str.contains('Новая', case=False, na=False)].shape[0]
            else:
                completed_requests = 0
                in_progress_requests = 0
                new_requests = 0
            
            info_text = f"Дата генерации: {datetime.now().strftime('%d.%m.%Y %H:%M')}\n"
            info_text += f"Всего заявок: {df.shape[0]}\n"
            info_text += f"Уникальных типов техники: {df['tech_type'].nunique() if 'tech_type' in df.columns else 0}\n"
            info_text += f"Уникальных моделей: {df['tech_model'].nunique() if 'tech_model' in df.columns else 0}\n"
            info_text += f"Мастеров в системе: {df['master_fio'].nunique() if 'master_fio' in df.columns else 0}\n"
            info_text += f"Клиентов в системе: {df['client_fio'].nunique() if 'client_fio' in df.columns else 0}\n\n"
            info_text += f"Статусы заявок:\n"
            info_text += f"• Готово к выдаче/Завершено: {completed_requests}\n"
            info_text += f"• В процессе ремонта: {in_progress_requests}\n"
            info_text += f"• Новые заявки: {new_requests}"
            
            ax.text(0.5, 0.35, info_text, 
                   fontsize=12, 
                   ha='center', va='center', transform=ax.transAxes,
                   bbox=dict(boxstyle="round,pad=0.5", facecolor="#E6F3FF", alpha=0.8, edgecolor="#0066CC"))
            
            pdf.savefig(fig, bbox_inches='tight')
            plt.close(fig)
            print("  ✓ Титульная страница создана")
            
            # 1. Круговая диаграмма: Распределение заявок по типам техники
            print("  Создание диаграммы 1: Распределение заявок по типам техники...")
            fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 7))
            
            if 'tech_type' in df.columns and df['tech_type'].notna().any():
                tech_type_counts = df['tech_type'].value_counts()
                
                # Улучшенная цветовая палитра - теплые тона
                colors1 = plt.cm.Set3(np.linspace(0.1, 0.9, len(tech_type_counts)))
                
                # Круговая диаграмма
                wedges, texts, autotexts = ax1.pie(tech_type_counts.values, 
                                                   labels=tech_type_counts.index,
                                                   autopct=lambda pct: f'{pct:.1f}%\n({int(pct/100*sum(tech_type_counts.values))})',
                                                   colors=colors1,
                                                   startangle=90,
                                                   textprops={'fontsize': 9},
                                                   wedgeprops={'edgecolor': 'white', 'linewidth': 1})
                
                ax1.set_title('Распределение заявок по типам техники', fontsize=14, fontweight='bold')
                
                # Гистограмма с улучшенной цветовой схемой
                colors2 = plt.cm.viridis(np.linspace(0.2, 0.8, len(tech_type_counts)))
                bars = ax2.bar(range(len(tech_type_counts)), tech_type_counts.values, 
                              color=colors2, edgecolor='black', linewidth=0.5)
                ax2.set_xticks(range(len(tech_type_counts)))
                ax2.set_xticklabels(tech_type_counts.index, rotation=45, ha='right', fontsize=9)
                ax2.set_title('Количество заявок по типам техники', fontsize=14, fontweight='bold')
                ax2.set_ylabel('Количество заявок')
                ax2.grid(axis='y', alpha=0.3, linestyle='--')
                
                # Добавляем значения на столбцы
                for bar in bars:
                    height = bar.get_height()
                    ax2.text(bar.get_x() + bar.get_width()/2., height + 0.1,
                            f'{int(height)}', ha='center', va='bottom', fontsize=9, fontweight='bold')
            else:
                ax1.text(0.5, 0.5, 'Нет данных о типах техники', 
                        ha='center', va='center', fontsize=12, color='gray')
                ax1.set_title('Распределение заявок по типам техники', fontsize=14, fontweight='bold')
                ax2.text(0.5, 0.5, 'Нет данных о типах техники', 
                        ha='center', va='center', fontsize=12, color='gray')
                ax2.set_title('Количество заявок по типам техники', fontsize=14, fontweight='bold')
            
            plt.tight_layout()
            pdf.savefig(fig, bbox_inches='tight')
            plt.close(fig)
            print("  ✓ Диаграмма 1 создана")
            
            # 2. Распределение заявок по статусам и времени в процессе
            print("  Создание диаграмма 2: Статусы заявок и время в процессе...")
            fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 7))
            
            if 'request_status' in df.columns and df['request_status'].notna().any():
                status_counts = df['request_status'].value_counts()
                
                # Улучшенная цветовая палитра для статусов
                status_colors = {
                    'Готова к выдаче': '#4CAF50',  # Зеленый
                    'В процессе ремонта': '#FF9800',  # Оранжевый
                    'Новая заявка': '#2196F3',  # Синий
                    'Завершена': '#8BC34A'  # Светло-зеленый
                }
                
                colors = [status_colors.get(status, plt.cm.tab20c(i/len(status_counts))) 
                         for i, status in enumerate(status_counts.index)]
                
                # Круговая диаграмма статусов с тенью
                wedges, texts, autotexts = ax1.pie(status_counts.values, 
                                                   labels=status_counts.index,
                                                   autopct=lambda pct: f'{pct:.1f}%\n({int(pct/100*sum(status_counts.values))})',
                                                   colors=colors,
                                                   startangle=90,
                                                   explode=[0.05 if i == status_counts.values.argmax() else 0 for i in range(len(status_counts))],
                                                   shadow=True,
                                                   textprops={'fontsize': 9})
                
                ax1.set_title('Распределение заявок по статусам', fontsize=14, fontweight='bold')
                
                # Box plot времени в процессе (для завершенных заявок)
                if 'days_in_process' in df.columns and df['days_in_process'].notna().any():
                    # Отфильтруем только заявки с заполненным временем в процессе
                    valid_days = df[df['days_in_process'].notna() & (df['days_in_process'] > 0)]
                    if len(valid_days) > 0:
                        # Группируем по статусам
                        status_list = []
                        data_to_plot = []
                        
                        for status in ['Готова к выдаче', 'В процессе ремонта', 'Новая заявка']:
                            if status in valid_days['request_status'].values:
                                status_list.append(status)
                                data_to_plot.append(valid_days[valid_days['request_status'] == status]['days_in_process'].values)
                        
                        if data_to_plot:
                            bp = ax2.boxplot(data_to_plot, patch_artist=True, showmeans=True, 
                                           meanline=True, showfliers=False,
                                           medianprops={'color': 'red', 'linewidth': 2},
                                           meanprops={'color': 'blue', 'linewidth': 1.5, 'linestyle': '--'})
                            
                            # Цвета для box plot
                            box_colors = [status_colors.get(status, plt.cm.tab20c(i/len(status_list))) 
                                         for i, status in enumerate(status_list)]
                            
                            for patch, color in zip(bp['boxes'], box_colors):
                                patch.set_facecolor(color)
                                patch.set_alpha(0.7)
                            
                            ax2.set_xticklabels(status_list, rotation=45, ha='right', fontsize=9)
                            ax2.set_ylabel('Дней в обработке', fontsize=10)
                            ax2.set_title('Время обработки по статусам заявок', fontsize=14, fontweight='bold')
                            ax2.grid(True, alpha=0.3, linestyle='--')
                        else:
                            ax2.text(0.5, 0.5, 'Нет данных для box plot', 
                                    ha='center', va='center', fontsize=12, color='gray')
                            ax2.set_title('Время обработки по статусам заявок', fontsize=14, fontweight='bold')
                    else:
                        ax2.text(0.5, 0.5, 'Нет данных о времени обработки', 
                                ha='center', va='center', fontsize=12, color='gray')
                        ax2.set_title('Время обработки по статусам заявок', fontsize=14, fontweight='bold')
                else:
                    ax2.text(0.5, 0.5, 'Нет данных о времени обработки', 
                            ha='center', va='center', fontsize=12, color='gray')
                    ax2.set_title('Время обработки по статусам заявок', fontsize=14, fontweight='bold')
            else:
                ax1.text(0.5, 0.5, 'Нет данных о статусах заявок', 
                        ha='center', va='center', fontsize=12, color='gray')
                ax1.set_title('Распределение заявок по статусам', fontsize=14, fontweight='bold')
                ax2.text(0.5, 0.5, 'Нет данных о статусах заявок', 
                        ha='center', va='center', fontsize=12, color='gray')
                ax2.set_title('Время обработки по статусам заявок', fontsize=14, fontweight='bold')
            
            plt.tight_layout()
            pdf.savefig(fig, bbox_inches='tight')
            plt.close(fig)
            print("  ✓ Диаграмма 2 создана")
            
            # 3. Анализ по мастерам
            print("  Создание диаграммы 3: Анализ работы мастеров...")
            fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 7))
            
            if 'master_fio' in df.columns and df['master_fio'].notna().any():
                # Количество заявок по мастерам
                master_counts = df['master_fio'].value_counts().head(10)  # Топ-10 мастеров
                
                # Гистограмма заявок по мастерам с градиентной окраской
                colors = plt.cm.plasma_r(np.linspace(0.2, 0.9, len(master_counts)))
                bars = ax1.barh(range(len(master_counts)), master_counts.values,
                               color=colors, edgecolor='black', linewidth=0.5, height=0.7)
                ax1.set_yticks(range(len(master_counts)))
                
                # Обрезаем длинные ФИО
                labels = [name[:20] + '...' if len(name) > 20 else name for name in master_counts.index]
                ax1.set_yticklabels(labels, fontsize=9)
                ax1.invert_yaxis()
                ax1.set_xlabel('Количество заявок', fontsize=10)
                ax1.set_title('Топ-10 мастеров по количеству заявок', fontsize=14, fontweight='bold')
                ax1.grid(axis='x', alpha=0.3, linestyle='--')
                
                # Добавляем значения на столбцы
                for i, (bar, value) in enumerate(zip(bars, master_counts.values)):
                    ax1.text(value + 0.1, bar.get_y() + bar.get_height()/2,
                            f'{value}', ha='left', va='center', fontsize=9, fontweight='bold')
                
                # Статусы заявок по мастерам (для топ-5)
                if 'request_status' in df.columns and df['request_status'].notna().any():
                    top_5_masters = master_counts.head(5).index
                    master_status_data = []
                    
                    for master in top_5_masters:
                        master_df = df[df['master_fio'] == master]
                        if 'request_status' in master_df.columns:
                            status_counts_master = master_df['request_status'].value_counts()
                            master_status_data.append(status_counts_master)
                    
                    # Создаем stacked bar chart
                    if master_status_data:
                        # Получаем все уникальные статусы
                        all_statuses = pd.concat(master_status_data).index.unique()
                        
                        # Создаем матрицу для stacked bar chart
                        matrix = pd.DataFrame(index=top_5_masters, columns=all_statuses, data=0)
                        
                        for i, master in enumerate(top_5_masters):
                            master_df = df[df['master_fio'] == master]
                            if 'request_status' in master_df.columns:
                                status_counts = master_df['request_status'].value_counts()
                                for status, count in status_counts.items():
                                    matrix.loc[master, status] = count
                        
                        # Рисуем stacked bar chart
                        bottom = np.zeros(len(top_5_masters))
                        
                        # Цвета для статусов
                        status_color_map = {
                            'Готова к выдаче': '#4CAF50',
                            'В процессе ремонта': '#FF9800',
                            'Новая заявка': '#2196F3',
                            'Завершена': '#8BC34A'
                        }
                        
                        for i, status in enumerate(all_statuses):
                            color = status_color_map.get(status, plt.cm.Set2(i/len(all_statuses)))
                            ax2.bar(range(len(top_5_masters)), matrix[status].values, 
                                   bottom=bottom, label=status, color=color, edgecolor='white', linewidth=0.5)
                            bottom += matrix[status].values
                        
                        ax2.set_xticks(range(len(top_5_masters)))
                        ax2.set_xticklabels([name[:12] + '...' if len(name) > 12 else name for name in top_5_masters], 
                                           rotation=45, ha='right', fontsize=9)
                        ax2.set_ylabel('Количество заявок', fontsize=10)
                        ax2.set_title('Распределение статусов заявок по топ-5 мастерам', fontsize=14, fontweight='bold')
                        ax2.grid(axis='y', alpha=0.3, linestyle='--')
                        ax2.legend(title='Статус', fontsize=8, title_fontsize=9, loc='upper right')
                    else:
                        ax2.text(0.5, 0.5, 'Нет данных о статусах по мастерам', 
                                ha='center', va='center', fontsize=12, color='gray')
                        ax2.set_title('Распределение статусов заявок по топ-5 мастерам', fontsize=14, fontweight='bold')
                else:
                    ax2.text(0.5, 0.5, 'Нет данных о статусах', 
                            ha='center', va='center', fontsize=12, color='gray')
                    ax2.set_title('Распределение статусов заявок по топ-5 мастерам', fontsize=14, fontweight='bold')
            else:
                ax1.text(0.5, 0.5, 'Нет данных о мастерах', 
                        ha='center', va='center', fontsize=12, color='gray')
                ax1.set_title('Топ-10 мастеров по количеству заявок', fontsize=14, fontweight='bold')
                ax2.text(0.5, 0.5, 'Нет данных о мастерах', 
                        ha='center', va='center', fontsize=12, color='gray')
                ax2.set_title('Распределение статусов заявок по топ-5 мастерам', fontsize=14, fontweight='bold')
            
            plt.tight_layout()
            pdf.savefig(fig, bbox_inches='tight')
            plt.close(fig)
            print("  ✓ Диаграмма 3 создана")
            
            # 4. Анализ по клиентам
            print("  Создание диаграммы 4: Анализ клиентов...")
            fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 7))
            
            if 'client_fio' in df.columns and df['client_fio'].notna().any():
                # Количество заявок по клиентам
                client_counts = df['client_fio'].value_counts().head(10)  # Топ-10 клиентов
                
                # Гистограмма заявок по клиентам
                colors = plt.cm.coolwarm(np.linspace(0.2, 0.8, len(client_counts)))
                bars = ax1.barh(range(len(client_counts)), client_counts.values,
                               color=colors, edgecolor='black', linewidth=0.5, height=0.7)
                ax1.set_yticks(range(len(client_counts)))
                
                # Обрезаем длинные ФИО
                labels = [name[:20] + '...' if len(name) > 20 else name for name in client_counts.index]
                ax1.set_yticklabels(labels, fontsize=9)
                ax1.invert_yaxis()
                ax1.set_xlabel('Количество заявок', fontsize=10)
                ax1.set_title('Топ-10 клиентов по количеству заявок', fontsize=14, fontweight='bold')
                ax1.grid(axis='x', alpha=0.3, linestyle='--')
                
                # Добавляем значения на столбцы
                for i, (bar, value) in enumerate(zip(bars, client_counts.values)):
                    ax1.text(value + 0.1, bar.get_y() + bar.get_height()/2,
                            f'{value}', ha='left', va='center', fontsize=9, fontweight='bold')
                
                # Типы техники по клиентам (для топ-5)
                if 'tech_type' in df.columns and df['tech_type'].notna().any():
                    top_5_clients = client_counts.head(5).index
                    
                    # Создаем сводную таблицу
                    pivot_data = df[df['client_fio'].isin(top_5_clients)].pivot_table(
                        index='client_fio',
                        columns='tech_type',
                        values='request_id',
                        aggfunc='count',
                        fill_value=0
                    )
                    
                    # Нормализуем для каждого клиента
                    pivot_data_norm = pivot_data.div(pivot_data.sum(axis=1), axis=0)
                    
                    # Heatmap
                    im = ax2.imshow(pivot_data_norm.values, cmap='YlOrRd', aspect='auto', vmin=0, vmax=1)
                    
                    # Настройка осей
                    ax2.set_xticks(np.arange(len(pivot_data_norm.columns)))
                    ax2.set_yticks(np.arange(len(pivot_data_norm.index)))
                    
                    # Укорачиваем названия для оси X
                    x_labels = [label[:15] + '...' if len(label) > 15 else label for label in pivot_data_norm.columns]
                    ax2.set_xticklabels(x_labels, rotation=45, ha='right', fontsize=8)
                    
                    # Укорачиваем названия для оси Y
                    y_labels = [label[:12] + '...' if len(label) > 12 else label for label in pivot_data_norm.index]
                    ax2.set_yticklabels(y_labels, fontsize=9)
                    
                    # Добавляем значения в ячейки
                    for i in range(len(pivot_data_norm.index)):
                        for j in range(len(pivot_data_norm.columns)):
                            value = pivot_data.iloc[i, j]
                            if value > 0:
                                text = ax2.text(j, i, f'{value}',
                                             ha="center", va="center",
                                             color="white" if pivot_data_norm.iloc[i, j] > 0.5 else "black",
                                             fontsize=8, fontweight='bold')
                    
                    ax2.set_title('Распределение типов техники по топ-5 клиентам', fontsize=14, fontweight='bold')
                    
                    # Цветовая шкала
                    cbar = plt.colorbar(im, ax=ax2, fraction=0.046, pad=0.04)
                    cbar.set_label('Доля заявок', fontsize=9)
                else:
                    ax2.text(0.5, 0.5, 'Нет данных о типах техники', 
                            ha='center', va='center', fontsize=12, color='gray')
                    ax2.set_title('Распределение типов техники по топ-5 клиентам', fontsize=14, fontweight='bold')
            else:
                ax1.text(0.5, 0.5, 'Нет данных о клиентах', 
                        ha='center', va='center', fontsize=12, color='gray')
                ax1.set_title('Топ-10 клиентов по количеству заявок', fontsize=14, fontweight='bold')
                ax2.text(0.5, 0.5, 'Нет данных о клиентах', 
                        ha='center', va='center', fontsize=12, color='gray')
                ax2.set_title('Распределение типов техники по топ-5 клиентам', fontsize=14, fontweight='bold')
            
            plt.tight_layout()
            pdf.savefig(fig, bbox_inches='tight')
            plt.close(fig)
            print("  ✓ Диаграмма 4 создана")
            
            # 5. Анализ временных показателей
            print("  Создание диаграммы 5: Временные показатели...")
            fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 7))
            
            if 'start_date' in df.columns and df['start_date'].notna().any():
                df['month_year'] = df['start_date'].dt.to_period('M').astype(str)
                monthly_counts = df.groupby('month_year').size()
                
                # График количества заявок по месяцам
                x_pos = np.arange(len(monthly_counts))
                colors = plt.cm.Blues(np.linspace(0.4, 0.9, len(monthly_counts)))
                
                bars = ax1.bar(x_pos, monthly_counts.values,
                              color=colors, edgecolor='black', linewidth=0.5)
                ax1.set_xticks(x_pos)
                ax1.set_xticklabels(monthly_counts.index, rotation=45, ha='right', fontsize=9)
                ax1.set_ylabel('Количество заявок', fontsize=10)
                ax1.set_title('Динамика заявок по месяцам', fontsize=14, fontweight='bold')
                ax1.grid(axis='y', alpha=0.3, linestyle='--')
                
                # Добавляем значения на столбцы
                for bar, value in zip(bars, monthly_counts.values):
                    height = bar.get_height()
                    ax1.text(bar.get_x() + bar.get_width()/2., height + 0.1,
                            f'{value}', ha='center', va='bottom', fontsize=9, fontweight='bold')
                
                # Время выполнения заявок (для завершенных)
                if 'completion_date' in df.columns and df['completion_date'].notna().any():
                    completed_df = df[df['completion_date'].notna()]
                    if len(completed_df) > 0:
                        completed_df['processing_days'] = (completed_df['completion_date'] - completed_df['start_date']).dt.days
                        
                        # Гистограмма времени выполнения
                        bins = min(15, len(completed_df['processing_days'].unique()))
                        n, bins, patches = ax2.hist(completed_df['processing_days'], bins=bins, 
                                                   color='#FFB74D', edgecolor='black', alpha=0.8)
                        ax2.set_xlabel('Дней на выполнение заявки', fontsize=10)
                        ax2.set_ylabel('Количество заявок', fontsize=10)
                        ax2.set_title('Распределение времени выполнения заявок', fontsize=14, fontweight='bold')
                        ax2.grid(True, alpha=0.3, linestyle='--')
                        
                        # Градиентная окраска столбцов гистограммы
                        for i, patch in enumerate(patches):
                            patch.set_facecolor(plt.cm.Oranges(i / len(patches)))
                        
                        # Добавляем среднее значение
                        if len(completed_df['processing_days']) > 0:
                            mean_days = completed_df['processing_days'].mean()
                            ax2.axvline(mean_days, color='red', linestyle='--', linewidth=2, 
                                      label=f'Среднее: {mean_days:.1f} дней')
                            ax2.legend(fontsize=9)
                    else:
                        ax2.text(0.5, 0.5, 'Нет завершенных заявок', 
                                ha='center', va='center', fontsize=12, color='gray')
                        ax2.set_title('Распределение времени выполнения заявок', fontsize=14, fontweight='bold')
                else:
                    ax2.text(0.5, 0.5, 'Нет данных о датах завершения', 
                            ha='center', va='center', fontsize=12, color='gray')
                    ax2.set_title('Распределение времени выполнения заявок', fontsize=14, fontweight='bold')
            else:
                ax1.text(0.5, 0.5, 'Нет данных о датах начала', 
                        ha='center', va='center', fontsize=12, color='gray')
                ax1.set_title('Динамика заявок по месяцам', fontsize=14, fontweight='bold')
                ax2.text(0.5, 0.5, 'Нет данных о датах начала', 
                        ha='center', va='center', fontsize=12, color='gray')
                ax2.set_title('Распределение времени выполнения заявок', fontsize=14, fontweight='bold')
            
            plt.tight_layout()
            pdf.savefig(fig, bbox_inches='tight')
            plt.close(fig)
            print("  ✓ Диаграмма 5 создана")
            
            # 6. Heatmap: Матрица тип техники vs статус
            print("  Создание диаграммы 6: Матрица тип техники vs статус...")
            fig, ax = plt.subplots(figsize=(12, 8))
            
            if 'tech_type' in df.columns and df['tech_type'].notna().any() and \
               'request_status' in df.columns and df['request_status'].notna().any():
                # Создаем сводную таблицу
                pivot_table = pd.pivot_table(df, 
                                            values='request_id',
                                            index='tech_type',
                                            columns='request_status',
                                            aggfunc='count',
                                            fill_value=0)
                
                # Нормализуем по строкам для лучшей визуализации
                pivot_table_norm = pivot_table.div(pivot_table.sum(axis=1), axis=0)
                
                # Heatmap
                im = ax.imshow(pivot_table_norm.values, cmap='YlOrRd', aspect='auto', vmin=0, vmax=1)
                
                # Настройка осей
                ax.set_xticks(np.arange(len(pivot_table.columns)))
                ax.set_yticks(np.arange(len(pivot_table.index)))
                ax.set_xticklabels(pivot_table.columns, rotation=45, ha='right', fontsize=9)
                ax.set_yticklabels(pivot_table.index, fontsize=9)
                
                # Добавляем значения в ячейки (абсолютные значения)
                for i in range(len(pivot_table.index)):
                    for j in range(len(pivot_table.columns)):
                        value = pivot_table.iloc[i, j]
                        if value > 0:
                            text = ax.text(j, i, f'{value}',
                                         ha="center", va="center",
                                         color="white" if pivot_table_norm.iloc[i, j] > 0.5 else "black",
                                         fontsize=9, fontweight='bold')
                
                ax.set_title('Матрица: Тип техники × Статус заявки', 
                            fontsize=14, fontweight='bold', pad=20)
                ax.set_xlabel('Статус заявки', fontsize=11)
                ax.set_ylabel('Тип техники', fontsize=11)
                
                # Цветовая шкала
                cbar = plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
                cbar.set_label('Доля заявок', fontsize=10)
            else:
                ax.text(0.5, 0.5, 'Нет данных для построения матрицы', 
                       ha='center', va='center', fontsize=12, color='gray')
                ax.set_title('Матрица: Тип техники × Статус заявки', 
                            fontsize=14, fontweight='bold', pad=20)
            
            plt.tight_layout()
            pdf.savefig(fig, bbox_inches='tight')
            plt.close(fig)
            print("  ✓ Диаграмма 6 создана")
            
            # 7. Сводная страница с таблицами
            print("  Создание сводной страницы с таблицами...")
            fig, axes = plt.subplots(2, 2, figsize=(14, 10))
            axes = axes.flatten()
            
            # Таблица 1: Топ-5 типов техники по количеству заявок
            ax = axes[0]
            ax.axis('tight')
            ax.axis('off')
            
            if 'tech_type' in df.columns and df['tech_type'].notna().any():
                top_5_tech = df['tech_type'].value_counts().head().reset_index()
                top_5_tech.columns = ['Тип техники', 'Количество заявок']
                
                # Создаем таблицу
                table1 = ax.table(cellText=top_5_tech.values,
                                 colLabels=top_5_tech.columns,
                                 cellLoc='center',
                                 loc='center',
                                 colWidths=[0.6, 0.4])
                table1.auto_set_font_size(False)
                table1.set_fontsize(9)
                table1.scale(1, 1.5)
                
                # Устанавливаем цвета для заголовка
                for i in range(len(top_5_tech.columns)):
                    table1[(0, i)].set_facecolor('#4A90E2')
                    table1[(0, i)].set_text_props(weight='bold', color='white')
                
                # Устанавливаем чередующиеся цвета для строк данных
                for i in range(1, len(top_5_tech) + 1):
                    color = '#F0F8FF' if i % 2 == 1 else '#E6F3FF'
                    for j in range(len(top_5_tech.columns)):
                        table1[(i, j)].set_facecolor(color)
                
                ax.set_title('Топ-5 типов техники', fontsize=12, fontweight='bold', y=0.95)
            else:
                ax.text(0.5, 0.5, 'Нет данных о типах техники', 
                       ha='center', va='center', fontsize=12, color='gray')
                ax.set_title('Топ-5 типов техники', fontsize=12, fontweight='bold', y=0.95)
            
            # Таблица 2: Статистика по статусам
            ax = axes[1]
            ax.axis('tight')
            ax.axis('off')
            
            if 'request_status' in df.columns and df['request_status'].notna().any():
                status_stats = df['request_status'].value_counts().reset_index()
                status_stats.columns = ['Статус', 'Количество']
                
                # Создаем таблицу
                table2 = ax.table(cellText=status_stats.values,
                                 colLabels=status_stats.columns,
                                 cellLoc='center',
                                 loc='center',
                                 colWidths=[0.6, 0.4])
                table2.auto_set_font_size(False)
                table2.set_fontsize(9)
                table2.scale(1, 1.5)
                
                # Устанавливаем цвета для заголовка
                for i in range(len(status_stats.columns)):
                    table2[(0, i)].set_facecolor('#4A90E2')
                    table2[(0, i)].set_text_props(weight='bold', color='white')
                
                # Устанавливаем чередующиеся цвета для строк данных
                for i in range(1, len(status_stats) + 1):
                    color = '#F0F8FF' if i % 2 == 1 else '#E6F3FF'
                    for j in range(len(status_stats.columns)):
                        table2[(i, j)].set_facecolor(color)
                
                ax.set_title('Распределение по статусам', fontsize=12, fontweight='bold', y=0.95)
            else:
                ax.text(0.5, 0.5, 'Нет данных о статусах', 
                       ha='center', va='center', fontsize=12, color='gray')
                ax.set_title('Распределение по статусам', fontsize=12, fontweight='bold', y=0.95)
            
            # Таблица 3: Статистика по времени обработки
            ax = axes[2]
            ax.axis('tight')
            ax.axis('off')
            
            time_stats_data = []
            
            # Среднее время в процессе
            if 'days_in_process' in df.columns and df['days_in_process'].notna().any():
                avg_time = df['days_in_process'].mean()
                time_stats_data.append(['Среднее время обработки', f'{avg_time:.1f} дн'])
            
            # Максимальное время в процессе
            if 'days_in_process' in df.columns and df['days_in_process'].notna().any():
                max_time = df['days_in_process'].max()
                time_stats_data.append(['Максимальное время', f'{max_time:.1f} дн'])
            
            # Минимальное время выполнения
            if 'completion_date' in df.columns and 'start_date' in df.columns:
                completed_df = df[df['completion_date'].notna()]
                if len(completed_df) > 0:
                    completed_df['processing_days'] = (completed_df['completion_date'] - completed_df['start_date']).dt.days
                    if completed_df['processing_days'].notna().any():
                        min_time = completed_df['processing_days'].min()
                        time_stats_data.append(['Минимальное время выполнения', f'{min_time} дн'])
            
            # Заявки в процессе более 30 дней
            if 'days_in_process' in df.columns and 'request_status' in df.columns:
                long_process = df[(df['days_in_process'] > 30) & 
                                 (df['request_status'].str.contains('В процессе', na=False))].shape[0]
                time_stats_data.append(['Заявок в процессе >30 дней', f'{long_process}'])
            
            if time_stats_data:
                # Создаем таблицу без заголовка
                table3 = ax.table(cellText=time_stats_data,
                                 cellLoc='left',
                                 loc='center',
                                 colWidths=[0.7, 0.3])
                table3.auto_set_font_size(False)
                table3.set_fontsize(9)
                table3.scale(1, 1.5)
                
                # Устанавливаем чередующиеся цвета для строк
                for i in range(len(time_stats_data)):
                    color = '#F0F8FF' if i % 2 == 0 else '#E6F3FF'
                    for j in range(2):  # 2 колонки
                        table3[(i, j)].set_facecolor(color)
                
                ax.set_title('Статистика времени', fontsize=12, fontweight='bold', y=0.95)
            else:
                ax.text(0.5, 0.5, 'Нет данных о времени обработки', 
                       ha='center', va='center', fontsize=12, color='gray')
                ax.set_title('Статистика времени', fontsize=12, fontweight='bold', y=0.95)
            
            # Таблица 4: Общая статистика
            ax = axes[3]
            ax.axis('tight')
            ax.axis('off')
            
            # Подготовка данных для таблицы
            total_stats_data = []
            
            # Всего заявок
            total_stats_data.append(['Всего заявок', f"{df.shape[0]}"])
            
            # Уникальные клиенты
            if 'client_fio' in df.columns:
                unique_clients = df['client_fio'].nunique()
                total_stats_data.append(['Уникальных клиентов', f"{unique_clients}"])
            else:
                total_stats_data.append(['Уникальных клиентов', 'Н/Д'])
            
            # Уникальные мастера
            if 'master_fio' in df.columns:
                unique_masters = df['master_fio'].nunique()
                total_stats_data.append(['Уникальных мастеров', f"{unique_masters}"])
            else:
                total_stats_data.append(['Уникальных мастеров', 'Н/Д'])
            
            # Уникальные типы техники
            if 'tech_type' in df.columns:
                unique_tech_types = df['tech_type'].nunique()
                total_stats_data.append(['Уникальных типов техники', f"{unique_tech_types}"])
            else:
                total_stats_data.append(['Уникальных типов техники', 'Н/Д'])
            
            # Заявки с комментариями
            if 'has_comment' in df.columns:
                comments_count = df['has_comment'].sum() if df['has_comment'].dtype in [bool, 'bool'] else 0
                total_stats_data.append(['Заявок с комментариями', f"{int(comments_count)}"])
            else:
                total_stats_data.append(['Заявок с комментариями', 'Н/Д'])
            
            # Заявки с запчастями
            if 'repair_parts' in df.columns:
                parts_count = df['repair_parts'].notna().sum()
                total_stats_data.append(['Заявок с запчастями', f"{parts_count}"])
            else:
                total_stats_data.append(['Заявок с запчастями', 'Н/Д'])
            
            # Среднее время в процессе
            if 'days_in_process' in df.columns and df['days_in_process'].notna().any():
                avg_days = df['days_in_process'].mean()
                total_stats_data.append(['Среднее время в процессе', f"{avg_days:.1f} дн"])
            else:
                total_stats_data.append(['Среднее время в процессе', 'Н/Д'])
            
            # Самый частый тип техники
            if 'tech_type' in df.columns and not df['tech_type'].mode().empty:
                most_common_tech = df['tech_type'].mode()[0]
                total_stats_data.append(['Самый частый тип техники', f"{most_common_tech[:20]}{'...' if len(most_common_tech) > 20 else ''}"])
            else:
                total_stats_data.append(['Самый частый тип техники', 'Н/Д'])
            
            # Создаем таблицу без заголовка
            table4 = ax.table(cellText=total_stats_data,
                             cellLoc='left',
                             loc='center',
                             colWidths=[0.6, 0.4])
            table4.auto_set_font_size(False)
            table4.set_fontsize(9)
            table4.scale(1, 1.5)
            
            # Устанавливаем чередующиеся цвета для строк
            for i in range(len(total_stats_data)):
                color = '#F0F8FF' if i % 2 == 0 else '#E6F3FF'
                for j in range(2):  # 2 колонки
                    table4[(i, j)].set_facecolor(color)
            
            ax.set_title('Общая статистика', fontsize=12, fontweight='bold', y=0.95)
            
            plt.suptitle('Сводные таблицы', fontsize=16, fontweight='bold', y=0.98)
            plt.tight_layout(rect=[0, 0, 1, 0.96])
            pdf.savefig(fig, bbox_inches='tight')
            plt.close(fig)
            print("  ✓ Сводная страница создана")
            
            # Страница с метаданными
            fig, ax = plt.subplots(figsize=(11, 8.5))
            ax.axis('off')
            
            metadata_text = "Метаданные отчета по сервисным заявкам\n\n"
            metadata_text += f"Исходный файл: {os.path.basename(excel_file_path)}\n"
            metadata_text += f"Дата генерации: {datetime.now().strftime('%d.%m.%Y %H:%M:%S')}\n"
            metadata_text += f"Всего страниц в отчете: {pdf.get_pagecount()}\n\n"
            metadata_text += f"Статистика данных:\n"
            metadata_text += f"• Всего заявок: {df.shape[0]}\n"
            metadata_text += f"• Столбцов: {df.shape[1]}\n"
            metadata_text += f"• Уникальных типов техники: {df['tech_type'].nunique() if 'tech_type' in df.columns else 0}\n"
            metadata_text += f"• Уникальных моделей: {df['tech_model'].nunique() if 'tech_model' in df.columns else 0}\n"
            metadata_text += f"• Уникальных мастеров: {df['master_fio'].nunique() if 'master_fio' in df.columns else 0}\n"
            metadata_text += f"• Уникальных клиентов: {df['client_fio'].nunique() if 'client_fio' in df.columns else 0}\n\n"
            metadata_text += "Содержание отчета:\n"
            metadata_text += "1. Распределение заявок по типам техники\n"
            metadata_text += "2. Статусы заявок и время в процессе\n"
            metadata_text += "3. Анализ работы мастеров\n"
            metadata_text += "4. Анализ клиентов\n"
            metadata_text += "5. Временные показатели\n"
            metadata_text += "6. Матрица тип техники vs статус\n"
            metadata_text += "7. Сводные таблицы\n\n"
            metadata_text += "Отчет создан автоматически с использованием:\n"
            metadata_text += "• Python pandas для анализа данных\n"
            metadata_text += "• Matplotlib и Seaborn для визуализации\n"
            metadata_text += "• Openpyxl для работы с Excel файлами\n"
            
            ax.text(0.1, 0.9, metadata_text, 
                   fontsize=12, 
                   va='top', transform=ax.transAxes,
                   bbox=dict(boxstyle="round,pad=1", facecolor="#F5F5F5", alpha=0.8, edgecolor="#CCCCCC"))
            
            pdf.savefig(fig, bbox_inches='tight')
            plt.close(fig)
            print("  ✓ Страница с метаданными создана")
        
        print(f"\n{'='*60}")
        print(f"✓ ОТЧЕТ УСПЕШНО СОЗДАН!")
        print(f"{'='*60}")
        print(f"Файл: {output_pdf_path}")
        print(f"Всего страниц: 8")
        print(f"Дата создания: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
        
        # Открываем файл (работает на Windows, Mac, Linux)
        try:
            import platform
            import subprocess
            
            system = platform.system()
            if system == 'Windows':
                os.startfile(output_pdf_path)
            elif system == 'Darwin':  # macOS
                subprocess.run(['open', output_pdf_path])
            else:  # Linux и другие
                subprocess.run(['xdg-open', output_pdf_path])
            print(f"✓ PDF файл открыт в программе для просмотра")
        except:
            print(f"⚠ Не удалось открыть файл автоматически. Откройте вручную: {output_pdf_path}")
        
        return output_pdf_path
    
    except FileNotFoundError:
        print(f"Ошибка: Файл {excel_file_path} не найден")
        raise
    except Exception as e:
        print(f"Ошибка при создании отчета: {str(e)}")
        import traceback
        traceback.print_exc()
        raise


def create_simple_service_report(excel_file_path, output_dir=None):
    """
    Создает упрощенный PDF отчет по сервисным заявкам
    
    Параметры:
    ----------
    excel_file_path : str
        Путь к Excel файлу с данными сервисных заявок
    output_dir : str, optional
        Директория для сохранения PDF файла
    
    Возвращает:
    -----------
    str
        Путь к созданному PDF файлу
    """
    
    # Определяем директорию для сохранения
    if output_dir is None:
        output_dir = os.getcwd()  # Текущая директория
    elif not os.path.exists(output_dir):
        os.makedirs(output_dir)
    
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    output_pdf_path = os.path.join(output_dir, f"service_report_{timestamp}.pdf")
    
    print(f"Создание отчета по сервисным заявкам...")
    print(f"Исходный файл: {excel_file_path}")
    print(f"Выходной файл: {output_pdf_path}")
    
    return create_service_dashboard_pdf(excel_file_path, output_pdf_path)


def fix_and_create_service_report():
    """
    Функция для создания отчета по сервисным заявкам
    """
    
    # Определяем путь к Excel файлу
    excel_file = 'service_requests_combined.xlsx'
    
    # Проверяем существование файла
    if not os.path.exists(excel_file):
        print(f"Ошибка: Файл {excel_file} не найден в текущей директории")
        print(f"Текущая директория: {os.getcwd()}")
        print(f"Содержимое директории:")
        for file in os.listdir('.'):
            if file.endswith('.xlsx'):
                print(f"  - {file}")
        return None
    
    try:
        # Создаем отчет в текущей директории с безопасным именем
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        output_file = f"service_requests_report_{timestamp}.pdf"
        
        print(f"Создание отчета из файла: {excel_file}")
        print(f"Отчет будет сохранен как: {output_file}")
        
        # Запускаем создание отчета
        pdf_path = create_service_dashboard_pdf(excel_file, output_file)
        
        print(f"\n✅ Отчет успешно создан!")
        print(f"📄 Файл: {pdf_path}")
        print(f"📁 Полный путь: {os.path.abspath(pdf_path)}")
        
        return pdf_path
        
    except Exception as e:
        print(f"❌ Ошибка при создании отчета: {e}")
        import traceback
        traceback.print_exc()
        return None


# Если скрипт запускается напрямую
if __name__ == "__main__":
    print("=" * 60)
    print("ГЕНЕРАТОР ОТЧЕТОВ ПО СЕРВИСНЫМ ЗАЯВКАМ")
    print("=" * 60)
    
    # Создаем отчет
    pdf_file = fix_and_create_service_report()
    
    if pdf_file:
        print(f"\n📊 Отчет успешно создан: {pdf_file}")
        print(f"📍 Файл сохранен в: {os.path.abspath(pdf_file)}")
        
        # Показываем инструкцию по открытию файла
        print("\n" + "=" * 60)
        print("ИНСТРУКЦИЯ ПО ОТКРЫТИЮ ФАЙЛА:")
        print("=" * 60)
        print("1. Найдите файл в проводнике:")
        print(f"   {os.path.abspath(pdf_file)}")
        print("\n2. Откройте файл двойным кликом")
        print("\n3. Или используйте команду в терминале:")
        print(f"   {pdf_file}")
    else:
        print("Не удалось создать отчет")

ГЕНЕРАТОР ОТЧЕТОВ ПО СЕРВИСНЫМ ЗАЯВКАМ
Создание отчета из файла: service_requests_combined.xlsx
Отчет будет сохранен как: service_requests_report_20251215_171922.pdf
Загрузка данных из файла: service_requests_combined.xlsx
Данные загружены: 5 строк, 23 столбцов
Столбцы: ['id', 'request_id', 'start_date', 'tech_type', 'tech_model', 'problem_description', 'request_status', 'completion_date', 'days_in_process', 'repair_parts', 'has_comment', 'comment_message', 'master_id', 'master_fio', 'master_phone', 'master_login', 'master_type', 'client_id', 'client_fio', 'client_phone', 'client_login', 'client_type', 'comment_master_id']
PDF файл будет сохранен как: c:\Users\user\Downloads\Practice_second_work\service_requests_report_20251215_171922.pdf
Создание PDF файла...
  ✓ Титульная страница создана
  Создание диаграммы 1: Распределение заявок по типам техники...
  ✓ Диаграмма 1 создана
  Создание диаграмма 2: Статусы заявок и время в процессе...
  ✓ Диаграмма 2 создана
  Создание диаграммы 3: 